In [4]:
import pandas as pd
import numpy as np
import random 
import sys
sys.path.append('..')
import importlib
import datagit.local_connector
importlib.reload(datagit.local_connector)
from datagit.local_connector import store_metric



from sqlalchemy import create_engine, text
engine = create_engine('postgresql://sammyteillet@localhost:5432/dbt-dummy-project')

snapshot_table = "bookings_snapshot"
date_column = "booking_date"
unique_key = "unique_key"

text_query = f"""
SELECT DISTINCT dbt_valid_from FROM {snapshot_table}
UNION
SELECT DISTINCT dbt_valid_to FROM {snapshot_table};
"""
query = text(text_query)
df = pd.read_sql(query, engine)
print(df)

for index, row in df.iloc[::-1].iterrows():
    date = row['dbt_valid_from']
    print(f"Processing data for date: {date}")
    if(pd.isna(date)):
        date = pd.Timestamp.now()

    
    asOfQuery = f"""
    SELECT * FROM {snapshot_table}
    WHERE ('{date}' BETWEEN dbt_valid_from AND dbt_valid_to) OR
          (dbt_valid_to IS NULL AND dbt_valid_from <= '{date}');
    """

    # Fetch data that's valid for the given snapshot date
    data_as_of_date = pd.read_sql(text(asOfQuery), engine)

    data_as_of_date.replace({np.nan: 'NA'}, inplace=True)

    # Drop dbt columns and add date and unique_key columns
    data_as_of_date = data_as_of_date.drop(['dbt_scd_id', 'dbt_updated_at', 'dbt_valid_from', 'dbt_valid_to'], axis=1)
    data_as_of_date["date"] = data_as_of_date[date_column]
    data_as_of_date["unique_key"] = data_as_of_date[unique_key]
    data_as_of_date = data_as_of_date.astype(str)
    print(data_as_of_date)

    store_metric(metric_name=snapshot_table, metric_value=data_as_of_date)


              dbt_valid_from
0                        NaT
1 2023-10-25 14:40:13.772308
2 2023-10-25 14:41:27.131453
Processing data for date: 2023-10-25 14:41:27.131453
                                 unique_key booking_date metric_value  \
0      63228a3e-18e6-4cf8-b809-ab7bb4ec4f2f   2023-09-07         4.31   
1      92b3d319-968e-42f7-9c26-d397d118cf94   2023-01-08         8.17   
2      f9ce25a8-9104-45de-8d20-2b12350dad9a   2023-03-12         8.34   
3      73f57f5c-8763-4cc2-8e85-51c8aed210bd   2023-07-26         8.86   
4      c38f587a-dfc2-4be2-bc88-e0b6998eae7c   2023-05-21         1.15   
...                                     ...          ...          ...   
60095  f5546a15-2329-427a-a2c5-88db07923085   2023-01-22         7.94   
60096  f8234a54-2ab4-4dec-b659-e921742e36ee   2023-05-25         2.89   
60097  f856b3e5-c279-4057-8e8f-8a95011b6e62   2023-01-04         1.88   
60098  fd69e3f3-facb-4bc9-89b6-51b8a012a0cd   2022-11-20         7.42   
60099  fda643c5-7a9f-444f-8b

/Users/sammyteillet/Documents/Projects/DataDrift/data-drift/tools/datagit/datagit/dataframe_update_breakdown.py:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  step2 = pd.concat([step1, new_data[step1.columns]], axis=0)


Processing data for date: NaT
                                 unique_key booking_date metric_value  \
0      63228a3e-18e6-4cf8-b809-ab7bb4ec4f2f   2023-09-07         4.31   
1      92b3d319-968e-42f7-9c26-d397d118cf94   2023-01-08         8.17   
2      f9ce25a8-9104-45de-8d20-2b12350dad9a   2023-03-12         8.34   
3      73f57f5c-8763-4cc2-8e85-51c8aed210bd   2023-07-26         8.86   
4      c38f587a-dfc2-4be2-bc88-e0b6998eae7c   2023-05-21         1.15   
...                                     ...          ...          ...   
59995  f5546a15-2329-427a-a2c5-88db07923085   2023-01-22         7.94   
59996  f8234a54-2ab4-4dec-b659-e921742e36ee   2023-05-25         2.89   
59997  f856b3e5-c279-4057-8e8f-8a95011b6e62   2023-01-04         1.88   
59998  fd69e3f3-facb-4bc9-89b6-51b8a012a0cd   2022-11-20         7.42   
59999  fda643c5-7a9f-444f-8b9c-725a4634dc3b   2023-01-18         5.07   

      country_code                  created_at                  updated_at  \
0              

/Users/sammyteillet/Documents/Projects/DataDrift/data-drift/tools/datagit/datagit/dataframe_update_breakdown.py:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  step2 = pd.concat([step1, new_data[step1.columns]], axis=0)
